# Environment Config

need to ensure that anaconda is installed. 

In [29]:
import pandas as pd
from PIL import Image
import os
from pathlib import Path
import shutil
from torchreid.utils import load_pretrained_weights

In [ ]:
# # cd to your preferred directory and clone this repo
# os.system('git clone https://github.com/KaiyangZhou/deep-person-reid.git')

# # create environment
# os.system('cd deep-person-reid/')
# os.system('conda create --name torchreid python=3.8')
# os.system('conda activate torchreid')

# # install dependencies
# # make sure `which python` and `which pip` point to the correct path
# os.system('pip install -r requirements.txt')

# # install torch and torchvision (select the proper cuda version to suit your machine)
# os.system('conda install pytorch torchvision cudatoolkit=9.0 -c pytorch')

# # install torchreid (don't need to re-build it if you modify the source code)
# os.system('python setup.py develop')

# Set up

In [1]:
import torchreid
from torchreid.data.datasets.image.robo import Robo
torchreid.data.register_image_dataset('robo', Robo)

/notebooks/deep-person-reid/torchreid/metrics/rank.py:11: UserWarning: Cython evaluation (very fast so highly recommended) is unavailable, now use python evaluation.
  warnings.warn(


In [2]:
datamanager = torchreid.data.ImageDataManager(
    root='reid-data',
    sources='robo',
    transforms = "",
    use_gpu = False
)

Building train transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
TRAIN:::  [('', 0, 0)]
QUERY:::  [('/notebooks/deep-person-reid/reid-data/sus/suspect.png', 0, 0), ('/notebooks/deep-person-reid/reid-data/sus/hostage.png', 0, 0)]
GALLERY:::  [('/notebooks/deep-person-reid/reid-data/test/0.png', 0, 2), ('/notebooks/deep-person-reid/reid-data/test/2.png', 0, 2), ('/notebooks/deep-person-reid/reid-data/test/1.png', 0, 2)]
=> Loaded Robo
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |     1 |        1 |         1
  query    |     1 |        2 |         1
  gallery  |     1 |        3 |         1
  ---------------------------

# Assemble Model

## model - osnet ibn x1_0

In [3]:
model = torchreid.models.build_model(
    name="osnet_ibn_x1_0",
    num_classes=datamanager.num_train_pids,
    loss="triplet",
    pretrained=False # I DISABLED DOWNLOAD WEIGHTS!!!
)

# model = model.cuda()

optimizer = torchreid.optim.build_optimizer(
    model,
    optim="adam",
    lr=0.0003
)

scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler="single_step",
    stepsize=20
)

# Assemble engine, load weights

In [4]:
engine = torchreid.engine.ImageTripletEngine(
    datamanager,
    model,
    optimizer=optimizer,
    scheduler=scheduler,
    label_smooth=True,
    use_gpu=False
)

In [5]:
from torchreid.utils import load_pretrained_weights
weight_path = '/notebooks/deep-person-reid/log/til_osnet_ibn_x1_0_with_pretrained/model/model.pth.tar-10-95.7-460'
load_pretrained_weights(model, weight_path)

Successfully loaded pretrained weights from "/notebooks/deep-person-reid/log/til_osnet_ibn_x1_0_with_pretrained/model/model.pth.tar-10-95.7-460"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']


In [32]:
# _,distmat, queries_and_galley = engine.run(
#     save_dir="log/til_resnet50",
#     max_epoch=4,
#     eval_freq=1,
#     print_freq=50,
#     test_only=True
# )

# Robotics challenge function

def targets_from_image(self, scene_img, target_img) -> BoundingBox:
        '''Process image with re-id pipeline and return the detected objects and their classes.
        
        Parameters
        ----------
        img : Any
            Input image.
        
        Returns
        -------
        results  : List[DetectedObject]
            List of DetectedObjects.
        '''
        
        # dummy data
        bbox = BoundingBox(100,100,300,50)
        obj = DetectedObject("1", "1", bbox)
        # DetectedObject: ['id', 'cls', 'bbox'] 
        # e.g. cls=0 means match class 0, cls=1 means match class 1, cls=2 means match class2.
        pass
        

In [10]:
def crop_image(image_path, results_csv, candidate_save_to):
    df = pd.read_csv(results_csv)
    
    # Open the image using Pillow
    image = Image.open(image_path)
    w = image.width
    h = image.height
    
    # Ready to save, check directoreis
    if os.path.exists(candidate_save_to):
        shutil.rmtree(candidate_save_to)
    os.mkdir(candidate_save_to)
    
    # crop candidates
    for row in df.index:
        
        # obtain coordinates for one target
        ymin = df['ymin'][row]
        xmin = df['xmin'][row]
        ymax = df['ymax'][row]
        xmax = df['xmax'][row]
    
    
        # Compute x, y, width, height from params
        xmin = w * float(xmin)
        ymin = h * float(ymin) 
        xmax = w * float(xmax)
        ymax = h * float(ymax)

        # Crop the image using the provided coordinates
        cropped_image = image.crop((xmin, ymin, xmax, ymax))

        # Save the cropped image
        cropped_image.save(candidate_save_to + f"/{row}.png")
        

In [21]:
import pandas as pd
df = pd.read_csv("/notebooks/deep-person-reid/reid-data/testcsv.csv")
df.head(5)

,Image_ID,class,confidence,ymin,xmin,ymax,xmax
0,image_0001,0,0.96,0.595313,0.389062,0.948438,0.531250
1,image_0001,0,0.94,0.728125,0.151562,0.884375,0.270313
2,image_0001,0,0.92,0.737500,0.692187,0.881250,0.818750


In [41]:
def output_matches(d, q, index):
    distmat = d[index]
    query = q[0][index][0]
    gallery = q[1]
    
    temp = zip(distmat, gallery)
    # print(query)
    # im=Image.open(query)
    # im.show()
    
    output = {}
    for i in sorted(temp):
        print(query)
        if "suspect" in query:
            if i[0] < 455 and output == {}:
                print("suspect")
                output[int(i[1][0][-5])] = "suspect"
            else:
                output[int(i[1][0][-5])] = 0
        elif "hostage" in query:
            if i[0] < 455 and output == {}:
                print("hostage")
                output[int(i[1][0][-5])] = "hostage"
            else:
                output[int(i[1][0][-5])] = 0
                
    return output

In [55]:
def reid(scene_img, results_csv, candidate_save_to, engine): 
    df = pd.read_csv(results_csv)
    
    # process/crop images in the scene
    crop_image(image_path, results_csv, candidate_save_to)
    
    # target image should be stored in target path alrd
    
    # call test function, obtain matrix
    _,distmat, queries_and_galley = engine.run(
        save_dir="",
        max_epoch=4,
        eval_freq=1,
        print_freq=50,
        test_only=True
    )
    
    for i in range(2):
        output = output_matches(distmat, queries_and_galley, i)
        print(output)
        for j in output.keys():
            if df.iloc[j]['class'] == 0:
                df.at[j,"class"] = output[j]
    return df
    

In [56]:
image_path = "/notebooks/deep-person-reid/reid-data/image_0001.png"
results_csv = "/notebooks/deep-person-reid/reid-data/testcsv.csv"
save_to = "/notebooks/deep-person-reid/reid-data/test"


df = reid(image_path, results_csv, save_to, engine)

##### Evaluating robo (source) #####
Extracting features from query set ...
HELLO HERE ARE UR PIDS AND CAMIDS: 
[0 0] [0 0]
Done, obtained 2-by-512 matrix
Extracting features from gallery set ...
HELLO HERE ARE UR PIDS AND CAMIDS: 
[0 0 0] [2 2 2]
Done, obtained 3-by-512 matrix
Speed: 1.7427 sec/batch
Computing distance matrix with metric=euclidean ...
YOUR DISTMAT!!!:  (2, 3)
[ 4.4276297e+02 -1.2207031e-04  5.0981458e+02]
[785.43    740.6364  621.69855]
Computing CMC and mAP ...
Note: number of gallery samples is quite small, got 3
/notebooks/deep-person-reid/reid-data/sus/suspect.png
suspect
/notebooks/deep-person-reid/reid-data/sus/suspect.png
/notebooks/deep-person-reid/reid-data/sus/suspect.png
{1: 'suspect', 2: 0, 0: 0}
/notebooks/deep-person-reid/reid-data/sus/hostage.png
/notebooks/deep-person-reid/reid-data/sus/hostage.png
/notebooks/deep-person-reid/reid-data/sus/hostage.png
{0: 0, 1: 0, 2: 0}


In [57]:
df

,Image_ID,class,confidence,ymin,xmin,ymax,xmax
0,image_0001,0,0.96,0.595313,0.389062,0.948438,0.531250
1,image_0001,suspect,0.94,0.728125,0.151562,0.884375,0.270313
2,image_0001,0,0.92,0.737500,0.692187,0.881250,0.818750
